In [1]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
**Task Definition**
You are an **SMT expert** specializing in **Semantic Fusion**, a method for validating SMT solvers. Your task is to **automatically fuse two equally satisfiable SMT formulas** into a new formula
 while **preserving satisfiability.**

The formulas are written in **SMT-LIB syntax**. Your goal is to take two input formulas **ω1 and ω2**, perform **Semantic Fusion**, and return a **new formula ω12**.
If the two input formulas **ω1 and ω2** are both satisfiable, perform **SAT Fusion**. The **returned new formula ω12** should also be **satisfiable**.
If the two input formulas **ω1 and ω2** are both unsatisfiable, perform **UNSAT Fusion**. The **returned new formula ω12** should also be **unsatisfiable**.

### SMT-LIB Syntax Guidelines:
**The generated output must strictly conform to the SMT-LIB format and include**:

1.	Variable Declarations: Use declare-fun to declare variables with their types: (declare-fun <var> () <type>).
2.	Function Definitions: Use define-fun to define functions: (define-fun <name> (<arg1 type1> <arg2 type2> ...) <return-type> <body>).
3.	Assertions: Use assert to specify constraints: (assert <constraint>).
4.	Boolean Expressions:
	•	Logical and: (and <expr1> <expr2> ...)
	•	Logical or: (or <expr1> <expr2> ...)
	•	Logical not: (not <expr>)
5.	Arithmetic Operations: Use standard operators like +, -, *, /, and div for integers.
6.	Relational Operators: Use comparison operators like >, <, >=, <=, and = for numerical expressions.
7.	Equality and Disequality: Use = for equality and distinct for disequality: (distinct <expr1> <expr2> ...).
8.	Conditionals: Use ite for if-then-else expressions: (ite <condition> <then-branch> <else-branch>).
9.	Quantifiers: Use forall and exists for quantified expressions:
	•	Universal: (forall ((x <type>) ...) <body>)
	•	Existential: (exists ((x <type>) ...) <body>)
10.	Constants: Declare constants directly using their types:
	•	Integers: 1, -3
	•	Real numbers: 1.0, -2.5
	•	Booleans: true, false
11.	Parentheses Matching: Ensure all expressions are enclosed in properly matched parentheses.
12.	Comments: Use ; for single-line comments in SMT-LIB.

**Avoid invalid SMT-LIB constructs! Ensure all parentheses are correctly matched.**

**Pre-requisites**
1. Decide if the input formulas are **both satisfiable** or **both unsatisfiable**.
    Rules to Follow:
	1.	Analyze Constraints:
	    • Identify all declared variables and their types.
	    • Analyze each constraint (assertion) applied to the variables.
        • Keep in mind that the **not** operator negates a formula.
	    • Determine if there exists at least one possible assignment of values that satisfies all constraints.
	2.	Decision Criteria:
	    • SAT (Satisfiable): If at least one valid assignment exists that makes all assertions true.
	    • UNSAT (Unsatisfiable): If no possible assignment can satisfy all assertions simultaneously.

2. If they are both satisfiable, perform ONLY **SAT Fusion Steps**. If they are both unsatisfiable, perform ONLY **UNSAT Fusion Steps**.

**SAT Fusion Steps**:

1. Merge ω1 and ω2 into a single formula ω1 ∧ ω2, preserving original declarations and assertions.

2. Introduce a fresh variable z which is not present in either input formula.

3. Based on the datatypes of x and y from the table below, choose the fusion function and then derive the inversion functions rx(y, z) and ry(x, z) from it :
    

| Type      | Fusion Function f(x, y)                | r_x(y, z)                                |  r_y(x, z)                                     |
|-----------|----------------------------------------|------------------------------------------|------------------------------------------------|
| **Int**   |  x + y                                 | z - y                                    | z - x                                          |
|           |  x + c + y                             | z - c - y                                | z - c - x                                      |
|           |  x * y                                 | z div y                                  | z div x                                        |
|           |  c_1 * x + c_2 * y + c_3               | (z - c_2 * y - c_3) div c_1              | (z - c_1 * x - c_3) div c_2                    |
| **Real**  |  x + y                                 | z - y                                    | z - x                                          |
|           |  x + c + y                             | z - c - y                                | z - c - x                                      |
|           |  x * y                                 | z / y                                    | z / x                                          |
|           |  c_1 * x + c_2 * y + c_3               | (z - c_2 * y - c_3) / c_1                | (z - c_1 * x - c_3) / c_2                      |
| **String**|  x str++ y                             | str.substr(z, 0, str.len(x))             | str.substr(z, str.len(x), str.len(y))          |
|           |  x str++ y                             | str.substr z 0 (str.len x)               | str.replace z x ""                             |
|           |  x str++ c str++ y                     | str.substr(z, 0, str.len(x))             | str.replace(str.replace(z, x, ""), c, "")      |



4. Next, define the inversion functions. In the example they would be defined as followed:
        	•	rx(y, z)  (expression for x)
	        •	ry(x, z)  (expression for y)
5. Then, derive expressions for `rx(y, z)` and `ry(x, z)`.
    
6.	Substitute variables:
	•	Randomly replace **one** free occurrence of x in ω1 **with the explicit expression value** of rx(y, z), **NOT rx(y, z) itself**.
	•	Randomly replace **one** free occurrence of y in ω2 **with the explicit expression value** of ry(y, z), **NOT rx(y, z) itself**.

7. **DO NOT introduce new assertions**—the fused formula should be **syntactically valid SMT-LIB** without modifying logical constraints.

8. Do NOT add any new assert blocks to the conjoined formula!


**UNSAT Fusion Steps**:

1. Merge ω1 and ω2 into a single formula ω1 ⊥ ω2, preserving original declarations and assertions.

2. Perform steps 2 to 6 from **SAT Fusion** without any modifications.

3. Since random substitutions of free occurances of variables may render the fused formula satisfiable, we need **Fusion Constraints** to ensure that rx and ry recover x and y. 
    Add **z = f(x, y)**, **x = rx(y, z)** and **y = ry(x, z)**, with their explicit expression values to the **final fused formula** as **Fusion Constraints**.


###SAT Fusion Example
**Formula1(ω1)**:
(declare-fun x () Real)
(assert (or (> x 0) (> x 1)))

**Formula2(ω2)**:
(declare-fun y () Real)
(assert (or (< y 0) (< y 1)))

**Expected Output**

(declare-fun x () Real)
(declare-fun y () Real)
(declare-fun z () Real)

; Fusion: Introduce z = x * y
(assert (or 
    (or (> x 0) (> (z / y) 1)) ; Randomly replaced only one x with (z div y)
    (or (< y 0) (< (z / x) 1)) ; Randomly  replaced only one y with (z div x)
))

###UNSAT Fusion Example
**Formula1(ω1)**:
(declare-fun x () Int)
(assert (and (> x 0) (< x 0)))  ; This formula is trivially unsatisfiable

**Formula2(ω2)**:
(declare-fun y () Int)
(assert (and (> y 1) (< y 1)))  ; This formula is trivially unsatisfiable

**Expected Output**

(declare-fun x () Int)
(declare-fun y () Int)
(declare-fun z () Int)

; Fusion: Introduce z = x + y
(assert (= z (+ x y)))

; Fused version of Formula1 (ω1)
(assert (and (> (/ z y) 0) (< (/ z y) 0)))  ; Randomly replaced x with (z / y)

; Fused version of Formula2 (ω2)
(assert (and (> (/ z x) 1) (< (/ z x) 1)))  ; Randomly replaced y with (z / x)

; Fusion constraints to enforce the relationship
(assert (= x (- z y)))
(assert (= y (- z x)))


###Additional Constraints
- Provide the result after performing each step.
- After performing **SAT Fusion**, **No new assertions should be added**, only substitutions.
- After performing **UNSAT Fusion**, ONLY the **Fusion Constraints** should be added and nothing else.
- **Variable names must remain unchanged**, except for z.
- Refrain from inserting comments in the final fused formula.

Now, perform the described operation for these formulas:

        """
        ),
        (
            "human",
            """
ω1: {formula1}
ω2: {formula2}
"""
    ),
]
)

In [8]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    api_key="KM3PGpAYB10SjVRj5IH5mysWswYiMJ4N"
)
chain = prompt | llm

In [ ]:
from langchain_ollama import ChatOllama 

llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0
)

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq (
    model="mixtral-8x7b-32768",
    temperature=0,
    api_key="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",
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

### Formulas to test SAT Fusion

In [4]:
# Formulae from the paper
chain = prompt | llm
formula1 = """
( declare-fun x () Int )
( declare-fun w () Bool )

( assert (= x (- 1) ) )
( assert (= w (= x (- 1) ) ) )
( assert w )
"""

formula2 = """
( declare-fun y () Int )
( declare-fun v () Bool )

( assert (= v ( not (= y (- 1) ) ) ) )
( assert ( ite v false (= y (- 1) ) ) )
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

To perform Semantic Fusion on the given formulas ω1 and ω2, I will first determine if they are both satisfiable or both unsatisfiable. Then, I will follow the appropriate fusion steps.

ω1:
(declare-fun x () Int)
(declare-fun w () Bool)
(assert (= x (- 1)))
(assert (= w (= x (- 1))))
(assert w)

ω2:
(declare-fun y () Int)
(declare-fun v () Bool)
(assert (= v (not (= y (- 1)))))
(assert (ite v false (= y (- 1))))

Both formulas are satisfiable, so I will perform SAT Fusion.

1. Merge ω1 and ω2 into a single formula ω1 ∧ ω2, preserving original declarations and assertions:

(declare-fun x () Int)
(declare-fun w () Bool)
(declare-fun y () Int)
(declare-fun v () Bool)
(assert (= x (- 1)))
(assert (= w (= x (- 1))))
(assert (= v (not (= y (- 1)))))
(assert (ite v false (= y (- 1))))

2. Introduce a fresh variable z which is not present in either input formula:

(declare-fun x () Int)
(declare-fun w () Bool)
(declare-fun y () Int)
(declare-fun v () Bool)
(declare-fun z () Int)
(assert (= x (

In [5]:
# Propositional Logic (QF_BV)
formula1 = """
(set-logic QF_BV)
(declare-fun a () Bool)
(declare-fun b () Bool)
(declare-fun c () Bool)

(assert (or (and a b) (and (not a) c)))
(assert (or (not b) c))
(assert (xor a c))
(check-sat)
"""

formula2 = """
(set-logic QF_BV)
(declare-fun x () Bool)
(declare-fun y () Bool)
(declare-fun z () Bool)

(assert (or (and x y) (not z)))
(assert (or x (not y) z))
(assert (xor y z))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

To perform Semantic Fusion on the given formulas ω1 and ω2, we first need to check if they are both satisfiable or both unsatisfiable. In this case, both formulas are satisfiable, so we will perform SAT Fusion.

**Step 1: Merge ω1 and ω2 into a single formula ω1 ∧ ω2, preserving original declarations and assertions.**

(set-logic QF_BV)
(declare-fun a () Bool)
(declare-fun b () Bool)
(declare-fun c () Bool)
(declare-fun x () Bool)
(declare-fun y () Bool)
(declare-fun z () Bool)

(assert (or (and a b) (and (not a) c)))
(assert (or (not b) c))
(assert (xor a c))
(assert (or (and x y) (not z)))
(assert (or x (not y) z))
(assert (xor y z))

**Step 2: Introduce a fresh variable z1 which is not present in either input formula.**

(declare-fun z1 () Bool)

**Step 3: Choose the fusion function and then derive the inversion functions rx(y, z1) and ry(x, z1) from it.**

For Boolean datatypes, the fusion function is x ⊕ y. The inversion functions are:

rx(y, z1) = z1 ⊕ y
ry(x, z1) = z1 ⊕ x

**Ste

In [ ]:
# Linear Integer Arithmetic (QF_LIA)
formula1 = """
(set-logic QF_LIA)
(declare-const x Int)
(declare-const y Int)

(assert (> x 5))
(assert (< x 20))
(assert (> y (- x 3)))
(assert (< y (* 2 x)))
(check-sat)
"""

formula2 = """
(set-logic QF_LIA)
(declare-const a Int)
(declare-const b Int)

(assert (> a 0))
(assert (< a 50))
(assert (= (* a b) 100))
(assert (> b 1))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Bit-Vector Theory (QF_BV)
formula1 = """
(set-logic QF_BV)
(declare-fun x () (_ BitVec 8))
(declare-fun y () (_ BitVec 8))

(assert (= (bvadd x y) (_ bv25 8)))
(assert (bvult x (_ bv20 8)))
(assert (bvugt y (_ bv4 8)))
(check-sat)
"""

formula2 = """
(set-logic QF_BV)
(declare-fun p () (_ BitVec 8))
(declare-fun q () (_ BitVec 8))

(assert (= (bvmul p q) (_ bv40 8)))
(assert (bvult q (_ bv10 8)))
(assert (bvugt p (_ bv3 8)))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Nonlinear Integer Arithmetic (QF_NIA)
formula1 = """
(set-logic QF_NIA)
(declare-const x Int)
(declare-const y Int)

(assert (> x 2))
(assert (< x 10))
(assert (= (* x y) 20))
(assert (> y 1))
(check-sat)
"""

formula2 = """
(set-logic QF_NIA)
(declare-const a Int)
(declare-const b Int)

(assert (> a 3))
(assert (< a 15))
(assert (= (* a b) 30))
(assert (> b 2))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Arrays (QF_AUFBV)
formula1 = """
(set-logic QF_AUFBV)
(declare-const arr (Array Int Int))
(declare-const i Int)

(assert (> i 2))
(assert (= (select arr i) 42))
(assert (= (store arr 5 100) arr))
(check-sat)
"""

formula2 = """
(set-logic QF_AUFBV)
(declare-const arr2 (Array Int Int))
(declare-const j Int)

(assert (< j 10))
(assert (= (select arr2 j) 84))
(assert (= (store arr2 7 50) arr2))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Floating-Point Arithmetic (QF_FP)
formula1 = """
(set-logic QF_FP)
(declare-const x Float32)

(assert (fp.gt x (fp #b0 #b10000001 #b00000000000000000000000)))  ;; x > 2.0
(assert (fp.lt x (fp #b0 #b10000011 #b00000000000000000000000)))  ;; x < 8.0
(check-sat)
"""

formula2 = """
(set-logic QF_FP)
(declare-const y Float32)

(assert (fp.gt y (fp #b0 #b10000000 #b00000000000000000000000)))  ;; y > 1.0
(assert (fp.lt y (fp #b0 #b10000010 #b00000000000000000000000)))  ;; y < 4.0
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Uninterpreted Functions (QF_UF)
formula1 = """
(set-logic QF_UF)
(declare-fun f (Int) Int)

(assert (= (f 1) 2))
(assert (= (f 2) 3))
(assert (= (f 3) 4))
(check-sat)
"""

formula2 = """
(set-logic QF_UF)
(declare-fun g (Int) Int)

(assert (= (g 4) 5))
(assert (= (g 5) 6))
(assert (= (g 6) 7))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

### Formulas to test UNSAT Fusion

In [25]:
# Formulas from the paper
chain = prompt | llm
formula1 = """
( declare-fun x () Real )
( assert ( not (= (+ (+ 1.0 x ) 6.0) (+ 7.0 x ) ) ) )
(check-sat)
"""

formula2 = """
( declare-fun y () Real )
( declare-fun w () Real )
( declare-fun v () Real )

( assert ( and ( < y v ) ( >= w v )( < (/ w v ) 0) ( > y 0) ) )
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

Let's analyze the given formulas ω1 and ω2 to determine their satisfiability and then perform the appropriate fusion.

### Formula ω1:
```smtlib
(declare-fun x () Real)
(assert (not (= (+ (+ 1.0 x) 6.0) (+ 7.0 x))))
(check-sat)
```

### Formula ω2:
```smtlib
(declare-fun y () Real)
(declare-fun w () Real)
(declare-fun v () Real)
(assert (and (< y v) (>= w v) (< (/ w v) 0) (> y 0)))
(check-sat)
```

### Step 1: Analyze Constraints

#### ω1 Analysis:
- Variable: `x` of type `Real`
- Constraint: `(not (= (+ (+ 1.0 x) 6.0) (+ 7.0 x)))`

Simplify the constraint:
```smtlib
(not (= (+ 1.0 x 6.0) (+ 7.0 x)))
```
This simplifies to:
```smtlib
(not (= (7.0 + x) (7.0 + x)))
```
This is always false because `7.0 + x` is always equal to `7.0 + x`. Therefore, the negation of this equality is always true, making ω1 satisfiable.

#### ω2 Analysis:
- Variables: `y`, `w`, `v` of type `Real`
- Constraints:
  - `(< y v)`
  - `(>= w v)`
  - `(< (/ w v) 0)`
  - `(> y 0)`

To satisfy all constraints:
- `y` m